# DEA Coastlines raster generation <img align="right" src="https://github.com/GeoscienceAustralia/dea-notebooks/raw/develop/Supplementary_data/dea_logo.jpg">

This code conducts raster generation for DEA Coastlines:

* Load stack of all available Landsat 5, 7, 8 and 9 satellite imagery for a location using [ODC Virtual Products](https://docs.dea.ga.gov.au/notebooks/Frequently_used_code/Virtual_products.html)
* Convert each satellite image into a remote sensing water index (MNDWI)
* For each satellite image, model ocean tides into a grid based on exact time of image acquisition
* Interpolate tide heights into spatial extent of image stack
* Mask out high and low tide pixels by removing all observations acquired outside of 50 percent of the observed tidal range centered over mean sea level
* Combine tidally-masked data into annual median composites representing the most representative position of the coastline at approximately mean sea level each year

This is an interactive version of the code intended for prototyping; to run this analysis at scale, use the [command line tools](DEACoastlines_generation_CLI.ipynb).

---

## Getting started
Set working directory to top level of repo to ensure links work correctly:

In [1]:
cd ..

/home/jovyan/Robbi/dea-coastlines


### Load packages

First we import the required Python packages, then we connect to the database, and load the catalog of virtual products.

In [ ]:
pip install -r requirements.in --quiet

In [2]:
%matplotlib inline
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

import os
import sys
import dask
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from functools import partial
from datacube.utils.geometry import Geometry
from dea_tools.dask import create_local_dask_cluster
import odc.geo.xr

# Load DEA Coastlines and intertidal code
import coastlines.raster

# Connect to datacube
import datacube

dc = datacube.Datacube(app="DEACoastlines")

# Create local dask client for parallelisation
client = create_local_dask_cluster(return_client=True)

Client Scheduler: tcp://127.0.0.1:43783 Dashboard: /user/robbi.bishoptaylor@ga.gov.au/proxy/8787/status,Cluster Workers: 1 Cores: 31 Memory: 254.70 GB


## Setup


### Set analysis parameters

In [3]:
study_area = 7931
raster_version = 'testing'
start_year = 2021
end_year = 2021

# Load analysis params from config file
config = coastlines.raster.load_config(
    config_path='configs/dea_coastlines_config.yaml')

### Load supplementary data

In [4]:
# Albers grid cells used to process the analysis
gridcell_gdf = (
    gpd.read_file(config['Input files']['grid_path']).to_crs(
        epsg=4326).set_index('id'))
gridcell_gdf.index = gridcell_gdf.index.astype(int).astype(str)
gridcell_gdf = gridcell_gdf.loc[[str(study_area)]]

## Loading data
### Create spatiotemporal query
This establishes the spatial and temporal extent used to search for Landsat satellite data.


In [5]:
# Create query based on analysis gridcell extent
geopoly = Geometry(gridcell_gdf.iloc[0].geometry, crs=gridcell_gdf.crs)
query = {
    'geopolygon': geopoly.buffer(0.05),
    'time': (str(start_year - 1), str(end_year + 1)),  # 1999, 2021
    'dask_chunks': {
        'time': 1,
        'x': 2048,
        'y': 2048
    }
}

### Load satellite data as MNDWI
This step loads satellite data from Landsat 5, 7, 8 and 9, and returns the data as a cloud-masked array converted to the Modified Normalised Difference Water Index (MNDWI).
For Digital Earth Australia Coastlines, this is achieved using [ODC Virtual Products](https://docs.dea.ga.gov.au/notebooks/Frequently_used_code/Virtual_products.html).

In [6]:
# Load MNDWI virtual product
ds = coastlines.raster.load_water_index(
    dc,
    query,
    yaml_path=config["Virtual product"]["virtual_product_path"],
    product_name=config["Virtual product"]["virtual_product_name"],
    mask_terrain_shadow=False,
)
ds

<xarray.Dataset>
Dimensions:      (time: 190, y: 2245, x: 1744)
Coordinates:
  * time         (time) datetime64[ns] 2020-01-01T23:24:57.987242 ... 2022-07...
  * y            (y) float64 -3.71e+06 -3.71e+06 ... -3.777e+06 -3.777e+06
  * x            (x) float64 3.14e+05 3.14e+05 3.141e+05 ... 3.663e+05 3.663e+05
    spatial_ref  int32 32656
Data variables:
    mndwi        (time, y, x) float32 dask.array<chunksize=(1, 2048, 1744), meta=np.ndarray>
    ndwi         (time, y, x) float32 dask.array<chunksize=(1, 2048, 1744), meta=np.ndarray>
Attributes:
    crs:           epsg:32656
    grid_mapping:  spatial_ref

In [ ]:
ds = ds.isel(time=slice(0, 10)).ndwi
ds.load()

In [ ]:
from itertools import repeat

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

dim='time'

def func(x, y=1, z=1):
    return x + y + z


with ProcessPoolExecutor() as executor:

    # Apply func in parallel
#     to_iterate = [group for (i, group) in ds.groupby(dim)]
    to_iterate = [1, 2, 3]
    args = (5, 10)
    iter_args = (to_iterate, *(repeat(i, len(to_iterate)) for i in args))
    out_list = list(tqdm(executor.map(func, *iter_args), total=len(to_iterate)))



In [ ]:
out_list

In [ ]:
def func(x, *args):
    print(args)
    
func(5, 5, 10)

In [ ]:
to_iterate = [1, 2, 3]


(to_iterate, *[repeat(i, 3) for i in args])

In [ ]:
repeat(5, 3)

## Tidal modelling


### Interpolate tides into each satellite timestep
For each satellite timestep, model tide heights into a low-resolution 5 x 5 km grid (matching resolution of the FES2014 tidal model), then reproject modelled tides into the spatial extent of our satellite image. Add  this new data as a new variable in our satellite dataset to allow each satellite pixel to be analysed and filtered/masked based on the tide height at the exact moment of satellite image acquisition. 

In [15]:
ds["tide_m"], tides_lowres = coastlines.raster.pixel_tides(ds, resample_func=True)

Creating reduced resolution tide modelling array
Modelling tides using FES2014 tide model
Reprojecting tides into original array


100%|██████████| 190/190 [00:04<00:00, 38.95it/s]


Plot example interpolated tide surface for a single timestep:

In [ ]:
import matplotlib.pyplot as plt

# Plot
ds_i = ds['tide_m'].isel(time=-1).compute()
ds_i.plot.imshow(robust=True,
                 cmap='viridis',
                 size=12,
                 vmin=ds_i.min().item(),
                 vmax=ds_i.max().item())
gridcell_gdf.to_crs(ds.odc.geobox.crs).plot(ax=plt.gca(),
                                             facecolor='none',
                                             edgecolor='black');

### Calculate per-pixel tide cutoffs
Based on the entire time-series of tide heights, compute the max and min satellite-observed tide height for each pixel, then calculate tide cutoffs used to restrict our data to satellite observations centred over mid-tide (0 m Above Mean Sea Level).

In [ ]:
# Determine tide cutoff
tide_cutoff_min, tide_cutoff_max = coastlines.raster.tide_cutoffs(ds, tides_lowres, tide_centre=0.0)

## Generate yearly composites
Export tidally-masked MNDWI median composites for each year, and three-yearly composites used to gapfill poor data coverage areas.

In [ ]:
# If output folder doesn't exist, create it
output_dir = f'data/interim/raster/{raster_version}/{study_area}_{raster_version}'
os.makedirs(output_dir, exist_ok=True)

# Iterate through each year and export annual and 3-year gapfill composites
coastlines.raster.export_annual_gapfill(ds, output_dir, tide_cutoff_min,
                                        tide_cutoff_max)

### Close Dask client

In [ ]:
# client.close()

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** For assistance with any of the Python code or Jupyter Notebooks in this repository, please post a [Github issue](https://github.com/GeoscienceAustralia/dea-coastlines/issues/new).

**Last modified:** July 2022